ZADANIE3

INITIAL

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, avg, count, min, max, row_number

In [8]:
spark = SparkSession.builder.appName("WindowedAggregates").getOrCreate()

data_transactions = [
    (1, '2011-01-01', 500), (1, '2011-01-15', 50), (1, '2011-01-22', 250), 
    (1, '2011-01-24', 75), (1, '2011-01-26', 125), (1, '2011-01-28', 175),
    (2, '2011-01-01', 500), (2, '2011-01-15', 50), (2, '2011-01-22', 25), 
    (2, '2011-01-23', 125), (2, '2011-01-26', 200), (2, '2011-01-29', 250),
    (3, '2011-01-01', 500), (3, '2011-01-15', 50), (3, '2011-01-22', 5000), 
    (3, '2011-01-25', 550), (3, '2011-01-27', 95), (3, '2011-01-30', 2500)
]
columns_transactions = ["AccountId", "TranDate", "TranAmt"]
dataframe_transactions = spark.createDataFrame(data_transactions, columns_transactions)

windowSpec = Window.partitionBy("AccountId").orderBy("TranDate")

df_transactions = dataframe_transactions.withColumn("RunAvg", avg("TranAmt").over(windowSpec))
df_transactions = df_transactions.withColumn("RunTranQty", count("TranAmt").over(windowSpec))
df_transactions = df_transactions.withColumn("RunSmallAmt", min("TranAmt").over(windowSpec))
df_transactions = df_transactions.withColumn("RunLargeAmt", max("TranAmt").over(windowSpec))
df_transactions = df_transactions.withColumn("RunTotalAmt", sum("TranAmt").over(windowSpec))

df_transactions.show()

+---------+----------+-------+------------------+----------+-----------+-----------+-----------+
|AccountId|  TranDate|TranAmt|            RunAvg|RunTranQty|RunSmallAmt|RunLargeAmt|RunTotalAmt|
+---------+----------+-------+------------------+----------+-----------+-----------+-----------+
|        1|2011-01-01|    500|             500.0|         1|        500|        500|        500|
|        1|2011-01-15|     50|             275.0|         2|         50|        500|        550|
|        1|2011-01-22|    250| 266.6666666666667|         3|         50|        500|        800|
|        1|2011-01-24|     75|            218.75|         4|         50|        500|        875|
|        1|2011-01-26|    125|             200.0|         5|         50|        500|       1000|
|        1|2011-01-28|    175|195.83333333333334|         6|         50|        500|       1175|
|        2|2011-01-01|    500|             500.0|         1|        500|        500|        500|
|        2|2011-01-15|     50|

In [9]:
sliding_window = windowSpec.rowsBetween(-2, 0)
df_transactions = dataframe_transactions.withColumn("SlideAvg", avg("TranAmt").over(sliding_window))
df_transactions = df_transactions.withColumn("SlideQty", count("TranAmt").over(sliding_window))
df_transactions = df_transactions.withColumn("SlideMin", min("TranAmt").over(sliding_window))
df_transactions = df_transactions.withColumn("SlideMax", max("TranAmt").over(sliding_window))
df_transactions = df_transactions.withColumn("SlideTotal", sum("TranAmt").over(sliding_window))
df_transactions = df_transactions.withColumn("RN", row_number().over(windowSpec))

df_transactions.show()

+---------+----------+-------+------------------+--------+--------+--------+----------+---+
|AccountId|  TranDate|TranAmt|          SlideAvg|SlideQty|SlideMin|SlideMax|SlideTotal| RN|
+---------+----------+-------+------------------+--------+--------+--------+----------+---+
|        1|2011-01-01|    500|             500.0|       1|     500|     500|       500|  1|
|        1|2011-01-15|     50|             275.0|       2|      50|     500|       550|  2|
|        1|2011-01-22|    250| 266.6666666666667|       3|      50|     500|       800|  3|
|        1|2011-01-24|     75|             125.0|       3|      50|     250|       375|  4|
|        1|2011-01-26|    125|             150.0|       3|      75|     250|       450|  5|
|        1|2011-01-28|    175|             125.0|       3|      75|     175|       375|  6|
|        2|2011-01-01|    500|             500.0|       1|     500|     500|       500|  1|
|        2|2011-01-15|     50|             275.0|       2|      50|     500|    

In [6]:
data_logical = [
    (1, 'George', 800), (2, 'Sam', 950), (3, 'Diane', 1100),
    (4, 'Nicholas', 1250), (5, 'Samuel', 1250), (6, 'Patricia', 1300),
    (7, 'Brian', 1500), (8, 'Thomas', 1600), (9, 'Fran', 2450),
    (10, 'Debbie', 2850), (11, 'Mark', 2975), (12, 'James', 3000),
    (13, 'Cynthia', 3000), (14, 'Christopher', 5000)
]
columns_logical = ["RowID", "FName", "Salary"]
df_logical = spark.createDataFrame(data_logical, columns_logical)

sum_window = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_logical = df_logical.withColumn("SumByRows", sum("Salary").over(sum_window))
df_logical = df_logical.withColumn("SumByRange", sum("Salary").over(Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)))

df_logical.show()

+-----+-----------+------+---------+----------+
|RowID|      FName|Salary|SumByRows|SumByRange|
+-----+-----------+------+---------+----------+
|    1|     George|   800|      800|       800|
|    2|        Sam|   950|     1750|      1750|
|    3|      Diane|  1100|     2850|      2850|
|    4|   Nicholas|  1250|     4100|      5350|
|    5|     Samuel|  1250|     5350|      5350|
|    6|   Patricia|  1300|     6650|      6650|
|    7|      Brian|  1500|     8150|      8150|
|    8|     Thomas|  1600|     9750|      9750|
|    9|       Fran|  2450|    12200|     12200|
|   10|     Debbie|  2850|    15050|     15050|
|   11|       Mark|  2975|    18025|     18025|
|   12|      James|  3000|    21025|     24025|
|   13|    Cynthia|  3000|    24025|     24025|
|   14|Christopher|  5000|    29025|     29025|
+-----+-----------+------+---------+----------+

